# Imports

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, log_loss
from sklearn.linear_model import LogisticRegression

In [ ]:
# Import Data
from google.colab import drive
drive.mount('/content/drive')
# X_test = pd.read_csv('/content/drive/MyDrive/BTAI (1)/cleaned/testWord2Vec.csv')
# X_train = pd.read_csv('/content/drive/MyDrive/BTAI (1)/cleaned/trainWord2Vec.csv')
# X_val = pd.read_csv('/content/drive/MyDrive/BTAI (1)/cleaned/valWord2Vec.csv')

X_test = pd.read_csv('/content/drive/MyDrive/BTAI (1)/cleaned/testWord2Vec.csv')
X_train = pd.read_csv('/content/drive/MyDrive/BTAI (1)/cleaned/trainWord2Vec.csv')
X_val = pd.read_csv('/content/drive/MyDrive/BTAI (1)/cleaned/valWord2Vec.csv')


Mounted at /content/drive


In [ ]:
feature_list = list(X_train.select_dtypes(include = ['float64']).columns)
#assuming our feature list been refined
X = X_train.drop(['is_salient', 'sentence'], axis=1) #features
y = X_train['is_salient'] #label

In [ ]:
X

,tokenized
0,"['neither', 'iphone', 'apple', 'watch', 'recor..."
1,"['neither', 'iphone', 'apple', 'watch', 'recor..."
2,"['neither', 'iphone', 'apple', 'watch', 'recor..."
3,"['idea', '']"
4,"['idea', '']"
...,...
51478,"['support', 'team', 'look', '']"
51479,"['pls', 'provide', 'previously', 'requested', ..."
51480,"['pls', 'provide', 'previously', 'requested', ..."
51481,"['rp', 'http', '', 'tcoq1obduyadh']"


In [ ]:
# Scoring Function
def score(X_train, y_train, model):
  scoring_functions = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'), #might not be useful-check into this
    'recall': make_scorer(recall_score, average='macro'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True)  # works after adding multi_class, in this scenario a continous variable is being used as the label so  multiple classes are being made
  }
  for score_name, scoring_function in scoring_functions.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scoring_function, error_score='raise')
    mean_score = np.mean(scores)
    # print(f'{score_name.capitalize()} Score: {mean_score:.4f}')
    results += (f'{score_name.capitalize()} Score: {mean_score:.4f}')
  return results

# Bayesian Optimization

In [ ]:
!pip install -U bayesian-optimization
# lr_model = train_test_LR(X_train, X_test, y_train, y_test)
# !pip install numpy==1.21.6

In [ ]:
# from bayesian_optimization import BayesianOptimization
from bayes_opt import BayesianOptimization

### Define the objective function.
This function will evaluate the performance of a logistic regression model on the training set for a given set of hyperparameter values.


In [ ]:
def objective_function(C, solver, max_iter):
    solver = list(solver_mapping.keys())[int(solver)]
    model = LogisticRegression(C=C, solver=solver, max_iter=int(max_iter))
    model.fit(X, y)

    accuracy = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return np.mean(accuracy)

In [ ]:
solver_mapping = {
    'saga': 0,
    'newton-cg': 1,
    'lbfgs': 2,
    'liblinear': 3,
    'sag':4,
    'newton-cholesky':5
}
hyperparameters = {
  'C': (1e-5, 1e5),
  'solver': (0, len(solver_mapping) - 1),  # index range for solver
  'max_iter': (1, 1000000)
}

### Initialize the Bayesian optimization algorithm

In [ ]:
bo = BayesianOptimization(
  f=objective_function,
  verbose=2,
  pbounds=hyperparameters
)

### Run the Bayesian optimization algorithm & Print Results

In [ ]:
bo.maximize(init_points=5, n_iter=10)
best_hyperparameters = bo.max['params']
print(best_hyperparameters)

|   iter    |  target   |     C     | max_iter  |  solver   |
-------------------------------------------------------------


ValueError: ignored

## Results
The Function Returned `{'C': 93655.43819938875, 'max_iter': 818620.7753348298, 'solver': 0.6552264622762094}` - meaning that `C: 93655`, `max_iter: 818620` and `solver: saga` are the best hyper parameters!

# Logistic Regression model `train_test_LR`

In [ ]:
#logistic regression model implementing log loss
# def train_test_LR(X_train, X_test, y_train, y_test):
'''
Fit a Linear Regression classifier to the training data X_train, y_train.
Return the loss and accuracy of resulting predictions on the test set.
Parameters: imported from bayesian optimization
'''
#create the logreg model
model = LogisticRegression({'C': 93655, 'max_iter': 818620, 'solver': 'saga'})


#fit model to training
model.fit(X, y)


#make model predictions using predcit_proba(). This methods predicts two outcomes (T/F) of the
probability_predictions = model.predict_proba(X_test)


# 4. Compute the log loss on 'probability_predictions' and save the result to the variable 'l_loss' below
  # YOUR CODE HERE
l_loss = log_loss(y_test, probability_predictions)


# 5. Make predictions on the test data using the predict() method and assign the result to the
# variable 'class_label_predictions' below
  # YOUR CODE HERE
class_label_predictions = model.predict(X_test)


# 6. Compute the accuracy score on 'class_label_predictions' and save the result to the variable 'acc_score' below
  # YOUR CODE HERE
acc_score = accuracy_score(y_test, class_label_predictions)
print("done running")
# return l_loss, acc_score


InvalidParameterError: ignored

In [ ]:
# The mean squared error
print('\nModel Performance\n\nRMSE =   %.2f'
      % np.sqrt(mean_squared_error(y_test, probability_predictions)))
# The coefficient of determination: 1 is perfect prediction
print(' R^2 =   %.2f'
      % r2_score(y_test, probability_predictions))

NameError: ignored

# Grid Search

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# Define the hyperparameter space
hyperparameter_space = {
    'C': np.logspace(-5, 5, 10),
    'solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
    'max_iter': [1,10,100,1000,10000, 100000,1000000,1000000000]
}

In [ ]:
# Create a grid search object
grid_search = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=hyperparameter_space,
    cv=2,
    scoring='accuracy'
)

In [ ]:
# Fit the grid search object to the training data
grid_search.fit(X_train, y_train)
print('Done')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Done


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Get the best hyperparameters
best_hyperparameters = grid_search.best_params_

In [ ]:
# Print the best hyperparameters
print('Best hyperparameters:', best_hyperparameters)

Best hyperparameters: {'C': 1e-05, 'max_iter': 1, 'solver': 'lbfgs'}


## Results
The Function Returned `{'C': 1e-05, 'solver': lbfgs}` - these are the default lowest values, so something went wrong.